# load mat pts files into a parcels trajectory output netcdf

note that the mat pts files have a fuckton of rows that are just all nan, so the data is pretty bloated.

TODO: dont save the completely nan rows

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import scipy.io
import xarray as xr

import utils
from parcels_utils import arrays_to_particlefilenc

In [ ]:
def parse_time(time):
    d = "".join(str(time).split("T")[0].split("-"))
    t = "".join(str(time).split("T")[1].split(":")[:2])
    return d, t

In [ ]:
start_time = np.datetime64("2020-08-10T00:30")
end_time = np.datetime64("2020-08-10T23:30")
incr = np.timedelta64(1, "h")

In [ ]:
dataset = None
current_time = start_time

while current_time <= end_time:
    date_parsed, time_parsed = parse_time(current_time)
    filename = f"pts_{date_parsed}_{time_parsed}.mat"
    if (utils.MATLAB_DIR / filename).is_file():
        pts_mat = scipy.io.loadmat(utils.MATLAB_DIR / filename)
    else:
        print(f"{utils.MATLAB_DIR / filename} not found")
        current_time += incr
        continue
    lon = np.array(pts_mat["xf"], dtype=np.float32)
    lat = np.array(pts_mat["yf"], dtype=np.float32)
    time = np.full((lon.size, 1), current_time)
    ds_curr = arrays_to_particlefilenc(time, lat, lon)
    if dataset is None:
        dataset = ds_curr
    else:
        dataset = xr.concat([dataset, ds_curr], dim="obs")
    current_time += incr

invalid rows are the rows where particles randomly disappear or reappear in the middle of the simulation

since I have no idea how the plume tracker code handles particle generation I have no idea how to interpret this

In [ ]:
invalid_rows = []
for i in range(dataset.dims["traj"]):
    num_real = (~np.isnan(dataset["lat"][i].values)).sum()
    if np.isnan(dataset["lat"][i, :num_real]).any():
        invalid_rows.append(i)
invalid_rows = np.array(invalid_rows)
invalid_rows

In [ ]:
date, time = parse_time(start_time)
dataset.to_netcdf(utils.PARTICLE_NETCDF_DIR / f"plume_pts_{date}_{time}.nc")